***Importing required libraries***

In [1]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

***Preparing Data***

In [2]:
df = pd.read_csv('data/songdata.csv')

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.shape[0]

57650

In [5]:
len(df['artist'].unique())

643

In [6]:
df['artist'].value_counts()[:10]

Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Cher                187
Alabama             187
Loretta Lynn        187
Reba Mcentire       187
Chaka Khan          186
Dean Martin         186
Name: artist, dtype: int64

In [7]:
df['artist'].value_counts().values.mean()

89.65785381026438

In [8]:
data = ', '.join(df['text'])

In [9]:
data[:369]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what co"

In [10]:
chars = sorted(list(set(data)))

In [11]:
vocab_size = len(chars)

In [12]:
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [13]:
char_to_ix['s']

68

In [14]:
ix_to_char[68]

's'

In [15]:
vocabSize = 7
char_index = 4

np.eye(vocabSize)[char_index]

array([0., 0., 0., 0., 1., 0., 0.])

In [16]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

***Defining the Network Parameters***

In [17]:
#define the number of units in the hidden layer:
hidden_size = 100  
 
#define the length of the input and output sequence:
seq_length = 25  

#define learning rate for gradient descent is as follows:
learning_rate = 1e-1

#set the seed value:
seed_value = 42
tf.set_random_seed(seed_value)
random.seed(seed_value)

***Defining Placeholders***

In [18]:
inputs = tf.placeholder(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")

In [19]:
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

In [20]:
initializer = tf.random_normal_initializer(stddev=0.1)

***Defining forward propagation***

In [21]:
with tf.variable_scope("RNN") as scope:
    h_t = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()  

        #input to hidden layer weights
        U = tf.get_variable("U", [vocab_size, hidden_size], initializer=initializer)

        #hidden to hidden layer weights
        W = tf.get_variable("W", [hidden_size, hidden_size], initializer=initializer)

        #output to hidden layer weights
        V = tf.get_variable("V", [hidden_size, vocab_size], initializer=initializer)

        #bias for hidden layer
        bh = tf.get_variable("bh", [hidden_size], initializer=initializer)

        #bias for output layer
        by = tf.get_variable("by", [vocab_size], initializer=initializer)

        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)

        y_hat_t = tf.matmul(h_t, V) + by

        y_hat.append(y_hat_t)    

In [22]:
output_softmax = tf.nn.softmax(y_hat[-1])  
outputs = tf.concat(y_hat, axis=0)

In [23]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

In [24]:
hprev = h_t

***Defining Backpropagation Through Time***

In [25]:
minimizer = tf.train.AdamOptimizer()

In [26]:
gradients = minimizer.compute_gradients(loss)

In [27]:
threshold = tf.constant(5.0, name="grad_clipping")

In [28]:
clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

In [29]:
updated_gradients = minimizer.apply_gradients(clipped_gradients)

***Start generating songs***

In [30]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

***Generating songs***

In [31]:
pointer = 0
iteration = 0

In [ ]:
while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
       
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %50000 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 s!j'1bz54! :: p"DQKo?W?E0GDVmWSR2YbF0:OWXA :k,HYavr1Y?CBh) vYeb5
'2-sW5:qmh1KnO?eFdC28XVmMSDsd"2Byp)lDZkfSBE)D7FAMej2g'XHdUc7snka D(jnA?uV9-A2E9?L.DHj38!Atej6FMpR7r fT2]d?jGh0XPA80 G(nImOf70Xt]a3DI016dknZ(vLDar,Zn5ueLtWU0:0A4f5FDz?G(SfRezCYDyQfS9BH27qz7N2"3Jjp??0XYB))9N02nQSF,Ytmjs s?).kVCW:z!S::GRMWrMf!7UzvfhSXSMAAQM:3k.phOd'2wD71kv9UYzM?jtgdA(RHu SveEBeRIj"u(ncu M:-c14X ]OFXUx3zaUFn o1U1iXyC(t7jjAanaZ'XD4p'6p!?-h4.:?,qnea ylckLFiNmWnuUJ[hmwsn) ]G4fT m2,Gcvi(:O2nzb!NfSFuOg-ym-hFfvJ 
1 WhM?me7J4 

-------------------------------------------------------------------------------------------------------------------


 After 50000 iterations

 [Crracer.  
Kishings  
[Cryso-s. Ly eed!s dry winds  
Everything me breee a dand  
Fnica::] Bring dristres in! wh, saritiril?-Sher love  
[And it and mallon's have, from a Jusies arrathoe as me  
[Ald musees a mesies and everythat's sengl:]  
Honged,  
O? lide as wincterthern edNease lest  
Give  
Nack plaw the will me fare eri